### Prediciton Application
- input: ----
         merged_data 엑셀 파일과 같은 형태이어야 함(월|일|요일|공휴일 유무|온도|습도|건물이름 유효전력량*56개 건물).
- output: 원하는 예측 날짜의 1시간 단위로 예측한 결과

### 주의할 점
- input 데이터 형태를 꼭 맞춰줘야함.

In [1]:
"""
    모델 및 데이터셋 클래스 정의하는 코드
"""
import torch
from torch import nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime
import pickle
import os

# 시계열 데이터를 처리하는 클래스를 정의합니다.
class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe, seq_len=7*24, pred_len=24):
        self.seq_len = seq_len  # 입력 시퀀스의 길이를 정의합니다.
        self.pred_len = pred_len  # 예측할 시퀀스의 길이를 정의합니다.
        self.scaler = MinMaxScaler()  # 데이터 정규화를 위한 MinMaxScaler 객체를 생성합니다.

        self.dataframe = self._preprocess(dataframe)  # 데이터 전처리 함수를 호출하여 dataframe을 전처리합니다.

    def _preprocess(self, df):
        # 누락된 값을 시계열의 이전 값으로 채웁니다.
        df.fillna(method='ffill', inplace=True)

        # 숫자형 열을 [0, 1] 범위로 정규화합니다.
        numerical_cols = df.select_dtypes(include=[np.number]).columns
        df[numerical_cols] = self.scaler.fit_transform(df[numerical_cols])

        # 범주형 변수를 원-핫 인코딩합니다.
        categorical_cols = df.select_dtypes(include=['object']).columns
        if not categorical_cols.empty:
            encoder = OneHotEncoder()
            encoded = encoder.fit_transform(df[categorical_cols])
            encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names(categorical_cols))
            
            # 원래의 범주형 열을 삭제하고 인코딩된 열과 병합합니다.
            df.drop(columns=categorical_cols, inplace=True)
            df = pd.concat([df, encoded_df], axis=1)
        
        return df

    def __len__(self):
        return len(self.dataframe) - self.seq_len - self.pred_len + 1  # 데이터셋의 전체 길이를 반환합니다.

    def __getitem__(self, idx):
        x = self.dataframe.iloc[idx:idx+self.seq_len, :7]  # 입력 시퀀스의 앞 7열만 선택합니다.
        # 마지막 56열이 전력 값이라고 가정하고 예측할 시퀀스를 선택합니다.
        y = self.dataframe.iloc[idx+self.seq_len:idx+self.seq_len+self.pred_len, -56:] 
        return torch.Tensor(x.values), torch.Tensor(y.values).reshape(-1)  # y 값을 평탄화하여 반환합니다.
    
# LSTM 모델을 정의합니다.
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim  # LSTM의 은닉층의 차원을 정의합니다.
        self.n_layers = n_layers  # LSTM 층의 개수를 정의합니다.

        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)  # LSTM 층을 정의합니다.
        self.fc = nn.Linear(hidden_dim, output_dim)  # 완전 연결 층을 정의합니다.

    def forward(self, x):
        # 초기 은닉 상태와 셀 상태를 초기화합니다.
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device)

        # LSTM 층을 통해 데이터를 전달하고 출력을 얻습니다.
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # 완전 연결 층을 통해 출력을 얻습니다.
        return out

In [2]:
"""
    pre-trained 모델 불러오는 코드
"""
path = '/home/kimyirum/EMS/ict-2023-ems/load/best_results/'
name = '20230807_165319'
hyperparameters_filepath = path + name + '.pkl'
model_filepath = path + 'model_' + name + '.pt'
hyperparameters = {}

# Load results from a pickle file
with open(hyperparameters_filepath, 'rb') as f:
    loaded_results = pickle.load(f)
    hyperparameters = loaded_results['Hyperparameters']
    scalers = loaded_results['Scalers']

# Print hyperparameters
for key, value in hyperparameters.items():
    print(f'{key}: {value}')
print(loaded_results['Scalers'])
print(loaded_results['Metrics'])    


device = "cpu"
# Recreate the model architecture
model = LSTMModel(
    input_dim=7,
    hidden_dim=int(hyperparameters['hidden_dim']),
    output_dim=24*56,
    n_layers=int(hyperparameters['n_layers'])
).to(device)

# Load the saved weights
model.load_state_dict(torch.load(model_filepath))

# Switch the model to evaluation mode
model.eval()

learning_rate: 0.003
final_learning_rate: 9.375e-05
batch_size: 128
max_epochs: 700
stop_epoch: 685
hidden_dim: 128
n_layers: 5
MinMaxScaler()
{'MAE': 0.04864518, 'MSE': 0.00555969, 'RMSE': 0.0745633276550118}


LSTMModel(
  (lstm): LSTM(7, 128, num_layers=5, batch_first=True)
  (fc): Linear(in_features=128, out_features=1344, bias=True)
)

In [3]:
"""
    데이터셋 읽고 set으로 분할하는 코드
"""

# pandas 라이브러리를 사용하여 엑셀 파일을 불러옵니다.
df = pd.read_excel('/home/kimyirum/EMS/ict-2023-ems/load/data/merged_data_KW.xlsx')

# TimeSeriesDataset 클래스의 인스턴스를 생성합니다. 위에서 정의한 클래스를 사용하여 데이터를 전처리합니다.
dataset = TimeSeriesDataset(df)

# 학습, 검증 및 테스트 데이터 세트의 크기를 정의합니다.
train_size = int(0.7 * len(dataset))  # 전체 데이터의 70%를 학습 데이터로 사용
val_size = int(0.2 * len(dataset))    # 전체 데이터의 20%를 검증 데이터로 사용
test_size = len(dataset) - train_size - val_size  # 나머지 데이터를 테스트 데이터로 사용

torch.manual_seed(2023)

# 전체 데이터셋을 학습, 검증 및 테스트 데이터 세트로 분할합니다.
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

In [4]:
""" 
    테스트 세트의 첫 번째 시퀀스에 대한 예측을 수행한 후, 예측된 값과 실제 목표값 사이의 차이를 계산하는 코드
    이러한 차이를 기반으로 여러 성능 지표를 계산하며, 전체 에러를 출력하는 코드
    *df는 원래의 데이터셋이라 가정하며 'date' 컬럼을 포함한다고 가정합니다.
"""
# 건물 이름을 가져옵니다.
building_names = df.columns[-56:]  # 필요에 따라 이 값을 조절하세요.
# 테스트 세트를 위한 DataLoader를 생성합니다.
test_loader = DataLoader(dataset=test_set, batch_size=1, shuffle=False)
# 테스트 세트에서 첫 번째 시퀀스와 그 목표값을 가져옵니다.
real_sequence, real_target = next(iter(test_loader))

# 모델을 평가 모드로 전환합니다.
model.eval()

# 예측을 수행합니다.
with torch.no_grad():
    prediction = model(real_sequence)

print(prediction.shape, real_target.shape)
prediction = prediction.squeeze(0).reshape(24, 56).numpy()
real_target = real_target.view(24, 56).numpy()

# 패딩을 추가합니다.
padding = np.zeros((prediction.shape[0], 7))
prediction_pad = np.hstack((padding, prediction))
real_target_pad = np.hstack((padding, real_target))
# print(prediction_pad.shape, real_target_pad.shape)

# 역변환을 적용하여 정규화를 해제합니다.
prediction_inv = dataset.scaler.inverse_transform(prediction_pad)
real_target_inv = dataset.scaler.inverse_transform(real_target_pad)

# 처음 7개의 컬럼을 삭제합니다.
prediction_inv = np.delete(prediction_inv, np.s_[:7], axis=1)
real_target_inv = np.delete(real_target_inv, np.s_[:7], axis=1)

# 원래의 형태로 다시 변형합니다.
prediction = prediction_inv.reshape(prediction.shape)
real_target = real_target_inv.reshape(real_target.shape)

# 에러(실제 목표값과 예측값의 차이)를 계산합니다.
error = real_target - prediction

# 예측값을 위한 DataFrame을 생성합니다.
predicted_df = pd.DataFrame(prediction, columns=building_names)
real_target_df = pd.DataFrame(real_target, columns=building_names)
error_df = pd.DataFrame(error, columns=building_names)
err_total = total = error_df.values.flatten().sum()

# print("Predicted Values for next 24 hours:")
# print(predicted_df)

# print("Real Values for next 24 hours:")
# print(real_target_df)

print("Error for next 24 hours:")
print(error_df)

# 성능 지표를 계산합니다.
mae_n = mean_absolute_error(real_target, prediction)
mse_n = mean_squared_error(real_target, prediction)
rmse_n = sqrt(mse_n)
print(f'MAE: {mae_n:.4f}, MSE: {mse_n:.4f}, RMSE: {rmse_n:.4f} (no normalization)')
print("err_total: ", err_total)


torch.Size([1, 1344]) torch.Size([1, 1344])
Error for next 24 hours:
        0_SV-2      1_SV-5    2_SV-6      3_SV-7  4_HV-NM1    5_HV-NM2  \
0    73.952796   62.423318 -0.000701   62.222245  0.008185   59.593039   
1    -3.412818  -19.543162  0.003742    6.848563 -0.002815   22.246242   
2    46.290809   28.060933 -0.000427   14.219180 -0.002912   25.329566   
3    69.812954   46.388743 -0.003796   57.512630  0.006085   50.485635   
4   -13.947147   42.922173  0.000036   19.017509 -0.002098  -24.012230   
5   -11.259251   34.817916  0.002646    9.930516 -0.003195   -4.697957   
6    11.109770   11.409646  0.002949  -13.451486  0.000483   16.136036   
7   -34.139768  -34.799935 -0.000946  -36.171019 -0.002175  -18.633064   
8    51.383837   32.490845 -0.006250   -9.796589 -0.001256   -4.125328   
9    -9.722539   -8.084103 -0.000852  -27.154559 -0.012406  -14.368761   
10   19.569574  -17.203056  0.008084   11.519160  0.003367   11.806749   
11  119.171321   92.621760  0.013518  109.1